In [1]:
def loaddataMT5(symbol, from_date, to_date):
    import MetaTrader5 as mt5
    from datetime import datetime
    import pandas as pd 

    # Kết nối tới MetaTrader 5
    if not mt5.initialize():
        print("Khởi tạo MT5 không thành công")
        mt5.shutdown()

    from_date_str = datetime.strptime(from_date, '%Y-%m-%d')
    to_date_str = datetime.strptime(to_date, '%Y-%m-%d')
    
    # Lấy dữ liệu OHLC cho cặp tiền symbol trong khoảng thời gian đã xác định
    ohlc_data = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_D1, from_date_str, to_date_str)
    # Ngắt kết nối với MT5
    mt5.shutdown()

    # Chuyển dữ liệu nhận được thành DataFrame và hiển thị
    data = pd.DataFrame(ohlc_data)
    data['time'] = pd.to_datetime(data['time'], unit='s')

    # ohlc_df.reset_index(inplace=True)

    data = data.rename(columns={'time': 'Datetime'})        
    data = data.rename(columns={'open': 'Open'})       
    data = data.rename(columns={'high': 'High'})       
    data = data.rename(columns={'low': 'Low'})       
    data = data.rename(columns={'close': 'Close'})       
    data = data.rename(columns={'tick_volume': 'Volume'})       

    data = pd.DataFrame(data, columns=['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume'])

    return data

In [2]:
def loaddataMT5_FromTo(symbol, from_date, to_date, timeframe):
    import MetaTrader5 as mt5
    from datetime import datetime
    import pandas as pd 

    # Kết nối tới MetaTrader 5
    if not mt5.initialize():
        print("Khởi tạo MT5 không thành công")
        mt5.shutdown()

    from_date_str = datetime.strptime(from_date, '%Y-%m-%d')
    to_date_str = datetime.strptime(to_date, '%Y-%m-%d')
    
    # Lấy dữ liệu OHLC cho cặp tiền symbol trong khoảng thời gian đã xác định
    ohlc_data = mt5.copy_rates_range(symbol, timeframe, from_date_str, to_date_str)
    # Ngắt kết nối với MT5
    mt5.shutdown()

    # Chuyển dữ liệu nhận được thành DataFrame và hiển thị
    data = pd.DataFrame(ohlc_data)
    data['time'] = pd.to_datetime(data['time'], unit='s')

    # ohlc_df.reset_index(inplace=True)

    data = data.rename(columns={'time': 'Datetime'})        
    data = data.rename(columns={'open': 'Open'})       
    data = data.rename(columns={'high': 'High'})       
    data = data.rename(columns={'low': 'Low'})       
    data = data.rename(columns={'close': 'Close'})       
    data = data.rename(columns={'tick_volume': 'Volume'})       

    data = pd.DataFrame(data, columns=['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume'])

    return data

In [3]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta
import schedule
import time
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# ##############################################Step 1: Lấy dữ liệu##############################################
symbol = 'USDJPY'
from_date = (datetime.now() - timedelta(days=5)).strftime('%Y-%m-%d')  # Lấy ngày hôm qua
to_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
timeframe = 1 # 1m # 0x4000
data = loaddataMT5_FromTo(symbol, from_date, to_date, timeframe)

data

,Datetime,Open,High,Low,Close,Volume
0,2024-01-16 17:00:00,146.745,146.785,146.716,146.717,159
1,2024-01-16 17:01:00,146.717,146.723,146.689,146.701,102
2,2024-01-16 17:02:00,146.699,146.713,146.668,146.671,81
3,2024-01-16 17:03:00,146.671,146.675,146.652,146.670,98
4,2024-01-16 17:04:00,146.669,146.707,146.659,146.703,92
...,...,...,...,...,...,...
4699,2024-01-19 23:55:00,148.135,148.138,148.134,148.136,20
4700,2024-01-19 23:56:00,148.135,148.140,148.135,148.136,15
4701,2024-01-19 23:57:00,148.136,148.146,148.097,148.097,20
4702,2024-01-19 23:58:00,148.102,148.155,148.102,148.155,2


# Lay 1 danh sach

In [6]:
# Import necessary modules
from ssi_fc_data import fc_md_client, model
import pandas as pd
import json
import configDataSSL

# Create a Market Data Client
client = fc_md_client.MarketDataClient(configDataSSL)

listStock = ['STB','VCB', 'ACB', 'TCB', 'TPB']  # Changed to a list for ordered processing

# Initialize an empty list for all stock data
all_stock_data = []

# Iterate over each stock in the list
for stock in listStock:
    req = model.daily_ohlc(stock, "05/09/2023", "10/09/2023")
    data_dict = client.daily_ohlc(configDataSSL, req)
    
    if 'data' in data_dict:
        data_list = data_dict['data']
        df = pd.DataFrame(data_list)
        all_stock_data.append(df)
    else:
        print(f"No data returned for {stock}")

# Concatenate all DataFrames in the list into a single DataFrame
all_stock_data_df = pd.concat(all_stock_data, ignore_index=True)

# Print or work with the all_stock_data_df DataFrame
print(all_stock_data_df)


   Symbol Market TradingDate  Time   Open   High    Low  Close    Volume  \
0     STB   HOSE  05/09/2023  None  32700  33150  32400  32700  22518900   
1     STB   HOSE  06/09/2023  None  32600  33050  32150  33050  23144500   
2     STB   HOSE  07/09/2023  None  33000  33250  32650  33000  20729300   
3     STB   HOSE  08/09/2023  None  32800  32950  32500  32500  18547100   
4     VCB   HOSE  05/09/2023  None  89100  90700  89100  90100   1942200   
5     VCB   HOSE  06/09/2023  None  90200  90200  89700  90200   2017600   
6     VCB   HOSE  07/09/2023  None  90000  90800  89500  89500   1765300   
7     VCB   HOSE  08/09/2023  None  89500  89500  89100  89500   1341900   
8     ACB   HOSE  05/09/2023  None  22700  23050  22650  22750   7626800   
9     ACB   HOSE  06/09/2023  None  22750  22950  22600  22950   4778100   
10    ACB   HOSE  07/09/2023  None  23000  23150  22750  22900   9854900   
11    ACB   HOSE  08/09/2023  None  22900  22950  22700  22750   5525800   
12    TCB   

# Lay 1 danh sach Forex

In [7]:
def loaddataMT5_FromTo_List(symbol, from_date, to_date, timeframe):
    import MetaTrader5 as mt5
    from datetime import datetime
    import pandas as pd 

    # Kết nối tới MetaTrader 5
    if not mt5.initialize():
        print("Khởi tạo MT5 không thành công")
        mt5.shutdown()

    from_date_str = datetime.strptime(from_date, '%Y-%m-%d')
    to_date_str = datetime.strptime(to_date, '%Y-%m-%d')
    
    # Lấy dữ liệu OHLC cho cặp tiền symbol trong khoảng thời gian đã xác định
    ohlc_data = mt5.copy_rates_range(symbol, timeframe, from_date_str, to_date_str)
    # Ngắt kết nối với MT5
    mt5.shutdown()

    # Chuyển dữ liệu nhận được thành DataFrame và hiển thị
    data = pd.DataFrame(ohlc_data)
    data['time'] = pd.to_datetime(data['time'], unit='s')

    # ohlc_df.reset_index(inplace=True)
    data['Symbol'] = symbol
    data = data.rename(columns={'time': 'Datetime'})        
    data = data.rename(columns={'open': 'Open'})       
    data = data.rename(columns={'high': 'High'})       
    data = data.rename(columns={'low': 'Low'})       
    data = data.rename(columns={'close': 'Close'})       
    data = data.rename(columns={'tick_volume': 'Volume'})       

    data = pd.DataFrame(data, columns=['Symbol', 'Datetime', 'Open', 'High', 'Low', 'Close', 'Volume'])

    return data

In [9]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta
import schedule
import time
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# ##############################################Step 1: Lấy dữ liệu##############################################
listsymbol = ['GBPUSD', 'USDJPY', 'EURUSD']
all_symbol_data = []

# Fetch data for each symbol and append to the list
for symbol in listsymbol:
    from_date = (datetime.now() - timedelta(days=5)).strftime('%Y-%m-%d')  # Lấy ngày hôm qua
    to_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
    timeframe = 1  # 1 minute timeframe
    data = loaddataMT5_FromTo_List(symbol, from_date, to_date, timeframe)
    all_symbol_data.append(data)  # Append the data to the list

# Concatenate all data into a single DataFrame
all_stock_data_df = pd.concat(all_symbol_data, ignore_index=True)

print(all_stock_data_df)  # Display the concatenated data

       Symbol            Datetime     Open     High      Low    Close  Volume
0      GBPUSD 2024-01-22 00:05:00  1.26854  1.26854  1.26854  1.26854       1
1      GBPUSD 2024-01-22 00:13:00  1.26851  1.26851  1.26851  1.26851       1
2      GBPUSD 2024-01-22 00:15:00  1.26932  1.26932  1.26932  1.26932       1
3      GBPUSD 2024-01-22 00:17:00  1.26936  1.26936  1.26936  1.26936       1
4      GBPUSD 2024-01-22 00:18:00  1.26928  1.26928  1.26928  1.26928       1
...       ...                 ...      ...      ...      ...      ...     ...
12117  EURUSD 2024-01-22 15:24:00  1.08884  1.08884  1.08866  1.08869      72
12118  EURUSD 2024-01-22 15:25:00  1.08869  1.08872  1.08860  1.08861      73
12119  EURUSD 2024-01-22 15:26:00  1.08861  1.08876  1.08854  1.08876      93
12120  EURUSD 2024-01-22 15:27:00  1.08875  1.08882  1.08871  1.08877      79
12121  EURUSD 2024-01-22 15:28:00  1.08878  1.08883  1.08864  1.08869      73

[12122 rows x 7 columns]


# Lay 1 danh sach Forex: Dung ham Common

In [10]:
import sys
sys.path.append('../Common')

import CommonMT5

import MetaTrader5 as mt5
from datetime import datetime, timedelta
import schedule
import time
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# ##############################################Step 1: Lấy dữ liệu##############################################
listsymbol = ['USDJPY', 'EURUSD']
all_symbol_data = []

# Fetch data for each symbol and append to the list
for symbol in listsymbol:
    from_date = (datetime.now() - timedelta(days=5)).strftime('%Y-%m-%d')  # Lấy ngày hôm qua
    to_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
    timeframe = 1  # 1 minute timeframe
    data = CommonMT5.CommonMT5.loaddataMT5_FromTo_List(symbol, from_date, to_date, timeframe)
    all_symbol_data.append(data)  # Append the data to the list

# Concatenate all data into a single DataFrame
all_stock_data_df = pd.concat(all_symbol_data, ignore_index=True)

print(all_stock_data_df)  # Display the concatenated data

       Symbol            Datetime       Open       High        Low      Close  \
0      USDJPY 2024-01-16 17:00:00  146.74500  146.78500  146.71600  146.71700   
1      USDJPY 2024-01-16 17:01:00  146.71700  146.72300  146.68900  146.70100   
2      USDJPY 2024-01-16 17:02:00  146.69900  146.71300  146.66800  146.67100   
3      USDJPY 2024-01-16 17:03:00  146.67100  146.67500  146.65200  146.67000   
4      USDJPY 2024-01-16 17:04:00  146.66900  146.70700  146.65900  146.70300   
...       ...                 ...        ...        ...        ...        ...   
11226  EURUSD 2024-01-22 15:28:00    1.08878    1.08883    1.08864    1.08867   
11227  EURUSD 2024-01-22 15:29:00    1.08866    1.08879    1.08866    1.08872   
11228  EURUSD 2024-01-22 15:30:00    1.08874    1.08874    1.08832    1.08837   
11229  EURUSD 2024-01-22 15:31:00    1.08840    1.08848    1.08824    1.08827   
11230  EURUSD 2024-01-22 15:32:00    1.08826    1.08828    1.08819    1.08819   

       Volume  
0         1